Note: This notebook assumes that "gmn_example1_extract_candidate_contrails.ipynb" and "gmn_example2_run_segmentation.ipynb" has been run.

Here we visualise the outputs of the previously ran scripts.

In [7]:
import sys
sys.path.append("..")

import os
from glob import glob
import json
from PIL import Image, ImageDraw
import imageio
import time

In [8]:
base_dir = '../data/gmn_extracted_flight_images/US0001'
circle_radius = 5

In [9]:
meta_data_files = glob(os.path.join(base_dir, '*/*/metadata/*.json'))

for meta_data_file in meta_data_files:
    flight_dir = os.path.dirname(os.path.dirname(meta_data_file))
    output_video_dir = os.path.join(flight_dir, 'debug')
    os.makedirs(output_video_dir, exist_ok=True)
    output_file = os.path.join(output_video_dir, os.path.basename(meta_data_file).split('.')[0] + '.mp4')

    if os.path.exists(output_file):
        print(f'{output_file} already exists, skipping...')
        continue

    start_time = time.time()
    
    images = []

    with open(meta_data_file, 'r') as f:
        data = json.load(f)

    # Visualise frames with flight
    for frame in data['frames_with_flight']:
        image = Image.open(frame['file_path']).convert('RGBA')
        draw  = ImageDraw.Draw(image)

        for x, y in zip(frame['advected_x_coords'], frame['advected_y_coords']):
            draw.ellipse([(x-circle_radius, y-circle_radius),
                            (x+circle_radius, y+circle_radius)],
                            fill=(0, 255, 0), outline=(0, 255, 0))

        draw.ellipse([(frame['x_coord']-circle_radius, frame['y_coord']-circle_radius),
                        (frame['x_coord']+circle_radius, frame['y_coord']+circle_radius)],
                        fill=(255, 0, 0), outline=(255, 0, 0))
        
        images.append(image)

    # Visualise frames after flight
    sam2_output = sorted(glob(os.path.join(flight_dir, 'sam2_output/*.png')))
    for idx, frame in enumerate(data['frames_after_flight']):
        image = Image.open(frame['file_path']).convert('RGBA')
        sam2_mask = Image.open(sam2_output[idx]).convert('RGBA')

        combined_image = Image.blend(image, sam2_mask, 0.1)

        draw = ImageDraw.Draw(combined_image)
        
        for x, y in zip(frame['advected_x_coords'], frame['advected_y_coords']):
            draw.ellipse([(x-circle_radius, y-circle_radius),
                            (x+circle_radius, y+circle_radius)],
                            fill=(0, 255, 0), outline=(0, 255, 0))

        images.append(combined_image)

    # Save as .mp4
    imageio.mimwrite(output_file, images, quality=8, fps=25)

    print(f'Processed {meta_data_file} in {time.time()-start_time:.2f}s')

Processed ../data/gmn_extracted_flight_images/US0001\20230801\A36A10_UPS881\metadata\A36A10_UPS881.json in 1.52s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A037B9_DAL339\metadata\A037B9_DAL339.json in 1.47s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A08E46_FDX1311\metadata\A08E46_FDX1311.json in 1.34s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A168AD_FDX1678\metadata\A168AD_FDX1678.json in 1.25s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A18403_AAL2864\metadata\A18403_AAL2864.json in 1.45s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A1EA70_SWA138\metadata\A1EA70_SWA138.json in 1.17s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A2E51D_WSN286\metadata\A2E51D_WSN286.json in 1.40s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A2E87E_FDX1167\metadata\A2E87E_FDX1167.json in 1.15s
Processed ../data/gmn_extracted_flight_images/US0001\20230802\A33D97_FFT2986\metadata\A3